In [1]:
!pip3 install elasticsearch

In [2]:
!pip3 install elasticsearch-dsl

In [3]:
import sys
sys.path.insert(0, '../../../BERT-FAQ/')

# import required libraries
from elasticsearch_dsl import Index, Document, Integer, Text, analyzer, Keyword, Double
from elasticsearch_dsl.connections import connections
from elasticsearch import Elasticsearch, helpers

from evaluation import get_relevance_label_df
from shared.utils import load_from_json
from shared.utils import dump_to_json
from shared.utils import make_dirs
from indexer import ingest_data
from indexer import QA
import logging

In [4]:
# get list of query answer pairs

query_answer_pairs_filepath = '../../../BERT-FAQ/data/FAQIR/query_answer_pairs.json'
relevance_label_df = get_relevance_label_df(query_answer_pairs_filepath)
faq_qa_pair_df = relevance_label_df[relevance_label_df['query_type'] == 'faq']
faq_qa_pairs = faq_qa_pair_df.T.to_dict().values()

In [5]:
# check relevance_label_df
relevance_label_df

,id,question,answer,query_type,label
0,1,"How to remove tree sap from car? Hi,I have som...",I have had good luck with commercial products ...,faq,1
1,2,How do I get a sticker off of a car window (p...,Goo Gone is a fantastic product (available at ...,faq,1
2,3,How to get rid of garbage disposal odor?,1. Inspect the black rubber cover that fits in...,faq,1
3,4,How to maintain a lawn mower?,"To properly maintain your lawnmover, you'll ne...",faq,1
4,5,How to fix vertical blinds? I can no longer re...,Click on the link and follow the steps.,faq,1
...,...,...,...,...,...
16319,16320,Cigarette smoke odor is so annoying. Some frie...,"Get a bucket, fold some cotton towels into it,...",user_query,1
16320,16321,I love smoking a pipe but the smell is quite d...,"Get a bucket, fold some cotton towels into it,...",user_query,1
16321,16322,What are the best tips for getting rid of the ...,"Get a bucket, fold some cotton towels into it,...",user_query,1
16322,16323,The weed smell is really hard to get rid of. D...,"Get a bucket, fold some cotton towels into it,...",user_query,1


In [6]:
# check faq_qa_pair_df
faq_qa_pair_df

,id,question,answer,query_type,label
0,1,"How to remove tree sap from car? Hi,I have som...",I have had good luck with commercial products ...,faq,1
1,2,How do I get a sticker off of a car window (p...,Goo Gone is a fantastic product (available at ...,faq,1
2,3,How to get rid of garbage disposal odor?,1. Inspect the black rubber cover that fits in...,faq,1
3,4,How to maintain a lawn mower?,"To properly maintain your lawnmover, you'll ne...",faq,1
4,5,How to fix vertical blinds? I can no longer re...,Click on the link and follow the steps.,faq,1
...,...,...,...,...,...
4308,4309,How do you remove other car paint rubs from yo...,Rubbing compound then put some wax on it...bug...,faq,1
4309,4310,how to replace bathroom fan?,If you didn't buy the fan yet you have to cons...,faq,1
4310,4311,How do you change the small down lights used i...,The halogen is made such that the actual lamp ...,faq,1
4311,4312,How do I know how much air to put in car tires...,"Wow, so many answers, so few correct. It amaz...",faq,1


In [7]:
# convert to list
faq_qa_pairs = list(faq_qa_pairs)

In [8]:
# check first 5 entries in faq_qa_pairs
faq_qa_pairs[:5]

[{'id': 1,
  'question': 'How to remove tree sap from car? Hi,I have some tree sap on my car, both older (pretty dry) and some newer from a redwood tree.Any tips on how to remove it?',
  'answer': 'I have had good luck with commercial products available at auto supply stores for just a few dollars. Below are some additional suggestions if you want to try DIY:',
  'query_type': 'faq',
  'label': 1},
 {'id': 2,
  'question': 'How do I get a sticker off of a car window  (plastic part)?  Nail polish remover seems risky?',
  'answer': "Goo Gone is a fantastic product (available at Target and most any hardware store or drug store) and works amazingly well on even pesky sticker residue.  Soak the area, and wipe it away.  It might take repeated applications, but you can remove the sticker without scratching, and without leaving a gooey residue.Note that Goo Gone can affect some surfaces, including some plastics, so you should be careful, especially if it is clear plastic.  I've used Goo Gone m

In [9]:
try:
    
    es = connections.create_connection(hosts=['localhost'])
    
    # Index data to Elasticsearch 
    index_name = "faqir"
    
    # Initialize index (only perform once)
    index = Index(index_name)

    # Define custom settings
    index.settings(
        number_of_shards=1,
        number_of_replicas=0
    )

    # Delete the index, ignore if it doesn't exist
    index.delete(ignore=404)

    # Create the index in Elasticsearch
    index.create()

    # Register a document with the index
    index.document(QA)

    # Ingest data to Elasticsearch
    ingest_data(faq_qa_pairs, es=es, index=index_name)

    print("Finished indexing {} records to {} index".format(len(faq_qa_pairs), index_name))

except Exception:
    logging.error('exception occured', exc_info=True)


100%|██████████| 4313/4313 [00:00<00:00, 47384.27it/s]


Finished indexing 4313 records to faqir index
